# Special Theory of Relativity

This notebook contains the programmatic verification for the **Special Theory of Relativity** entry from the THEORIA dataset.

**Entry ID:** special_relativity  
**Required Library:** sympy 1.12.0

## Description
Einstein's special theory of relativity revolutionized our understanding of space, time, mass, and energy. Built on two postulates - the relativity principle and the constancy of light speed - it derives the Lorentz transformations that describe how space and time coordinates change between inertial frames. The theory unifies space and time into spacetime, introduces time dilation and length contraction, establishes the relativistic energy-momentum relations, and reveals the famous mass-energy equivalence E = mc². These transformations ensure the invariance of the speed of light and the spacetime interval, forming the foundation of modern physics.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.12.0

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# ===============================
# Programmatic verification for Special Relativity
# Python 3.11, SymPy 1.12
# Mirrors the derivation steps with asserts.
# ===============================

# Symbols
A, B, D, E, v, c, x, t = sp.symbols('A B D E v c x t', real=True, finite=True)
c_pos = sp.symbols('c_pos', positive=True, real=True)
# We'll replace c by c_pos in places that require positivity.

# ------------------------------
# Steps 1-3: Linear map + origin worldline
# ------------------------------
x_p = A*x + B*t
t_p = D*x + E*t

# Origin of S' has x' = 0, and in S it moves as x = v t:
eq_origin = sp.Eq(x_p.subs(x, v*t), 0)
B_sol = sp.solve(eq_origin, B)[0]  # B = -A v
assert sp.simplify(B_sol + A*v) == 0

# ------------------------------
# Steps 4-7: Light-speed invariance for right- and left-moving pulses
# ------------------------------
# For x = ± c t, demand x' = ± c t'
eq_plus  = sp.Eq(x_p.subs({x: c*t, B: B_sol}),  c * t_p.subs({x: c*t, B: B_sol}))
eq_minus = sp.Eq(x_p.subs({x: -c*t, B: B_sol}), -c * t_p.subs({x: -c*t, B: B_sol}))
sol_DE = sp.solve([eq_plus, eq_minus], (D, E), dict=True)[0]
D_sol, E_sol = sol_DE[D], sol_DE[E]

# Expected from derivation: E = A and D = -A v / c^2
assert sp.simplify(E_sol - A) == 0
assert sp.simplify(D_sol + A*v/c**2) == 0

# Substitute back:
x_p2 = sp.simplify(x_p.subs({B: B_sol, D: D_sol, E: E_sol}))   # A (x - v t)
t_p2 = sp.simplify(t_p.subs({B: B_sol, D: D_sol, E: E_sol}))   # A (t - v x / c^2)

# ------------------------------
# Steps 9-12: Interval invariance ⇒ solve for A = γ
# ------------------------------
# 1D invariant interval: x'^2 - c^2 t'^2 = x^2 - c^2 t^2
interval_expr = sp.simplify(x_p2**2 - c**2*t_p2**2)
factorized = sp.simplify(interval_expr/(x**2 - c**2*t**2))

# Enforce factorized = 1 to preserve the interval ⇒ A^2 (1 - v^2/c^2) = 1
A_candidates = sp.solve(sp.Eq(factorized, 1), A)
# From derivation: A^2 (1 - v^2/c^2) = 1, so A = ± 1/sqrt(1 - v^2/c^2)
# Choose positive root A = 1/sqrt(1 - v^2/c^2) = γ
gamma = 1/sp.sqrt(1 - v**2/c_pos**2)

# Final Lorentz transform:
x_prime = sp.simplify(x_p2.subs(A, gamma))
t_prime = sp.simplify(t_p2.subs(A, gamma))
assert sp.simplify(x_prime - gamma*(x - v*t)) == 0
assert sp.simplify(t_prime - gamma*(t - v*x/c**2)).subs(c, c_pos) == 0

# y' = y, z' = z (no change transverse):
y, z, y_p, z_p = sp.symbols('y z y_p z_p', real=True)
y_p = y
z_p = z
assert y_p == y and z_p == z

# ------------------------------
# Steps 13-14: Time dilation and length contraction
# ------------------------------
# Time dilation: for Δx = 0 ⇒ Δt' = γ Δt
dt, dx = sp.symbols('dt dx', real=True)
dtp = sp.simplify(gamma*(dt - v*dx/c_pos**2))
assert sp.simplify(dtp.subs(dx, 0) - gamma*dt) == 0

# Length contraction: measure length of rod at rest in S, with Δx (endpoints simultaneous in S').
# Simultaneity in S': Δt' = 0 ⇒ 0 = γ (Δt - v Δx / c^2) ⇒ Δt = v Δx / c^2.
# Then Δx' = γ (Δx - v Δt) = γ (Δx - v*(v Δx / c^2)) = Δx / γ.
dt_simul = sp.simplify(v*dx/c_pos**2)
dxp = sp.simplify(gamma*(dx - v*dt_simul))
assert sp.simplify(dxp - dx/gamma) == 0

# ------------------------------
# Steps 15-29: Energy from work and γ identities
# ------------------------------
m = sp.symbols('m', positive=True, real=True)
gamma_v = 1/sp.sqrt(1 - v**2/c_pos**2)

# dγ/dv identity (Step 22):
dgamma_dv = sp.simplify(sp.diff(gamma_v, v))
assert sp.simplify(dgamma_dv - (v/c_pos**2)*gamma_v**3) == 0

# Identity v^2 γ^2 / c^2 + 1 = γ^2 (Step 25):
lhs = v**2*gamma_v**2/c_pos**2 + 1
rhs = gamma_v**2
assert sp.simplify(lhs - rhs) == 0

# Total relativistic energy and momentum:
E_rel = sp.simplify(gamma_v*m*c_pos**2)
p_rel = sp.simplify(gamma_v*m*v)

# Rest limits:
assert sp.simplify(E_rel.subs(v, 0) - m*c_pos**2) == 0
assert sp.simplify(p_rel.subs(v, 0)) == 0

# ------------------------------
# Steps 31-37: Energy–momentum relation
# ------------------------------
E2_minus_p2c2 = sp.simplify(E_rel**2 - (p_rel*c_pos)**2)
assert sp.simplify(E2_minus_p2c2 - (m*c_pos**2)**2) == 0

# ------------------------------
# Extra sanity checks
# ------------------------------
# γ at v = 0.6c is 1.25:
gamma_test = sp.simplify(gamma_v.subs(v, sp.Rational(3,5)*c_pos))
assert sp.simplify(gamma_test - sp.Rational(5,4)) == 0

# Low-velocity expansion: E ≈ mc^2 + (1/2) m v^2 + O(v^4)
series = sp.series(E_rel.expand(), v, 0, 3).removeO()
expected_series = m*c_pos**2 + sp.Rational(1,2)*m*v**2
assert sp.simplify(series - expected_series) == 0

print('All special relativity verifications passed ✔')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=special_relativity.json](https://theoria-dataset.org/entries.html?entry=special_relativity.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0